#### For Colab

In [ ]:
"""
function ClickConnect(){
    console.log("Working");
    document.querySelector("colab-toolbar-button").click() 
}
var i = setInterval(ClickConnect, 900000)
clearInterval(i)
"""

'\nfunction ClickConnect(){\n    console.log("Working");\n    document.querySelector("colab-toolbar-button").click() \n}\nvar i = setInterval(ClickConnect, 900000)\nclearInterval(i)\n'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
drive_dir = '/content/drive/My\ Drive/MAGMA:\ Summarization/'

In [ ]:
!pip install -q --upgrade ipython
!pip install -q --upgrade ipykernel

In [ ]:
!pip install transformers
!pip install datasets
!pip install rouge_score
!pip install wandb
!pip install sacrebleu

### **HuggingFace T5 for summarization**

https://huggingface.co/transformers/model_doc/t5.html#transformers.T5ForConditionalGeneration

#### **Demo**

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# initialize the model architecture and weights
t5 = T5ForConditionalGeneration.from_pretrained("t5-base")

# initialize the model tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained("t5-base")

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
article = """When Liana Barrientos was 23 years old, she got married in Westchester County, New York. A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband. Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other. In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage. Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the 2010 marriage license application, according to court documents. Prosecutors said the marriages were part of an immigration scam. On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further. After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002. All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say. Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages. Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted. The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali. Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force."""

print(article)

In [ ]:
print("model's max allowed length:")
t5_tokenizer.model_max_length

In [ ]:
inputs = t5_tokenizer.encode("summarize: " + article,
                          return_tensors = "pt",
                          max_length = 512, truncation = True)
print(inputs.shape)
print(t5_tokenizer.convert_ids_to_tokens([inputs[0,0]]))
print(t5_tokenizer.decode(inputs[0])[:110])

In [ ]:
print("model's configurations:")
t5.config

#### **Beam search**

Beam search can work very well in tasks where the length of the desired generation is more or less predictable as in machine translation or summarization - see Murray et al. (2018) and Yang et al. (2018)

We have seen that beam search heavily suffers from repetitive generation. This is especially hard to control with n-gram*- or other penalties in story generation

In [ ]:
"""
For tasks with long output sequences, we found improved performance from using
beam search (Sutskever et al., 2014). Specifically, we use a beam width of 4 and a
length penalty of α = 0.6 (Wu et al., 2016) for the WMT translation and CNN/DM
summarization tasks.
"""

outputs = t5.generate(inputs,
                      max_length = 200,
                      min_length = 100,
                      length_penalty = 0.6,
                      num_beams = 4,
                      num_return_sequences = 4,
                      early_stopping = True)

In [ ]:
print(outputs.shape)
for output in outputs:
    print("\n")
    output = output[output.nonzero()]
    print(len(output))
    print(t5_tokenizer.decode(output, skip_special_tokens=True))

#### **Sampling**

As argued in Ari Holtzman et al. (2019), high quality human language does not follow a distribution of high probability next words. In other words, as humans, we want generated text to surprise us and not to be boring/predictable.

While applying temperature can make a distribution less random, in its limit, when setting temperature --> 0, temperature scaled sampling becomes equal to greedy decoding and will suffer from the same problems as before.

In [ ]:
print("do_sample: ", t5.config.do_sample)
print("temperature: ", t5.config.temperature)
print("top-k: ", t5.config.top_k)
print("top-p: ", t5.config.top_p)

In [ ]:
outputs = t5.generate(inputs,
                      max_length = 200,
                      min_length = 100,
                      length_penalty = 5,
                      do_sample = True,
                      temperature = 2.0,
                      top_k = 100,
                      num_beams = 4,
                      num_return_sequences = 4,
                      early_stopping = True)

In [ ]:
print(outputs.shape)
for output in outputs:
    print("\n")
    output = output[output.nonzero()]
    print(len(output))
    print(t5_tokenizer.decode(output, skip_special_tokens=True))

#### **Fine Tuning, the hard way**

https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_summarization_wandb.ipynb

##### Init

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# initialize the model architecture and weights
t5 = T5ForConditionalGeneration.from_pretrained("t5-base")

# initialize the model tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained("t5-base")

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
from torch import cuda
device = "cuda" if cuda.is_available() else "cpu"
print(device)

In [ ]:
import wandb
wandb.login()

wandb.init(project="T5_fine_tuning_newssummary")

config = wandb.config          # Initialize config
config.TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
config.VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
config.TRAIN_EPOCHS = 2        # number of epochs to train (default: 10)
config.VAL_EPOCHS = 1
config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
config.SEED = 42               # random seed (default: 42)
config.MAX_LEN = t5_tokenizer.model_max_length
config.SUMMARY_LEN = 150 

# Set random seeds and deterministic pytorch for reproducibility
torch.manual_seed(config.SEED)
np.random.seed(config.SEED)
torch.backends.cudnn.deterministic = True

##### Preparing the data

In [ ]:
class CustomDataset(Dataset):
    
    def __init__(self, data, tokenizer, input_len, summ_len):
        self.tokenizer = tokenizer
        self.data = data
        self.input_len = input_len
        self.summ_len = summ_len
        self.article = self.data.article
        self.summary = self.data.summary
    
    def __len__(self):
        return len(self.summary)
    
    def __getitem__(self, idx):
        article = str(self.article[idx])
        article = ' '.join(article.split())
        summary = str(self.summary[idx])
        summary = ' '.join(summary.split())
        
        source = self.tokenizer.batch_encode_plus(
            [article],
            max_length = self.input_len,
            truncation = True,
            padding = 'max_length',
            return_tensors = 'pt')
        target = self.tokenizer.batch_encode_plus(
            [summary],
            max_length = self.summ_len,
            truncation = True,
            padding = 'max_length',
            return_tensors='pt')
        
        input_ids = source['input_ids'].squeeze()
        attention_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        
        return {
            'input_ids': input_ids.to(dtype=torch.long), 
            'attention_mask': attention_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long)
        }

In [ ]:
import pandas as pd

df = pd.read_csv(data_dir, encoding='latin-1')
df = df[['text', 'ctext']]
df.ctext = 'summarize: ' + df.ctext
df = df.rename(columns = {'text': 'summary', 'ctext': 'article'})
#df.dropna(axis=0, inplace=True)
print(df.head())

ratio = 0.8
train_dataset = df.sample(frac = ratio, random_state = config.SEED)
val_dataset = df.drop(train_dataset.index).reset_index(drop = True)
train_dataset = train_dataset.reset_index(drop = True)

print("\n\nFULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(val_dataset.shape))

training_set = CustomDataset(
    train_dataset,
    t5_tokenizer,
    config.MAX_LEN,
    config.SUMMARY_LEN)
val_set = CustomDataset(
    val_dataset,
    t5_tokenizer,
    config.MAX_LEN,
    config.SUMMARY_LEN)

In [ ]:
train_params = {
    'batch_size': config.TRAIN_BATCH_SIZE,
    'shuffle': True,
    'num_workers': 0
}
val_params = {
    'batch_size': config.VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
}

training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **val_params)

##### Train

In [ ]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    
    model.train()
    for _, data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        
        decoder_input_ids = y[:, :-1].contiguous()
        labels = y[:, 1:].clone().detach()
        labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        
        input_ids = data['input_ids'].to(device, dtype = torch.long)
        attention_mask = data['attention_mask'].to(device, dtype = torch.long)

        outputs = model(
            input_ids = input_ids,
            attention_mask = attention_mask,
            decoder_input_ids = decoder_input_ids,
            labels = labels)
        loss = outputs[0]
        logits = outputs[1]
        
        if _%10 == 0:
            #loss_fct = CrossEntropyLoss(ignore_index=-100)
            #loss = loss_fct(lm_logits.view(-1, lm_logits.size(-1)), labels.view(-1))
            wandb.log({"Training Loss": loss.item()})
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
t5 = t5.to(device)
optimizer = torch.optim.Adam(
    params = t5.parameters(),
    lr = config.LEARNING_RATE)

# Log metrics with wandb
wandb.watch(t5, log="all")

# Training loop
print('Initiating Fine-Tuning for the model on our dataset')
for epoch in range(config.TRAIN_EPOCHS):
    train(epoch, t5_tokenizer, t5, device, training_loader, optimizer)

##### Validate

In [ ]:
def validate(epoch, tokenizer, model, device, loader):
    
    model.eval()
    predictions = []
    references = []
    
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)        
            input_ids = data['input_ids'].to(device, dtype = torch.long)
            attention_mask = data['attention_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = input_ids,
                attention_mask = attention_mask,
                max_length = config.SUMMARY_LEN,
                num_beams = 2,
                repetition_penalty = 2.5,
                length_penalty = 1.0,
                early_stopping = True)
            
            preds = [tokenizer.decode(g, skip_special_tokens = True, clean_up_tokenization_spaces = True) for g in generated_ids]
            refs = [tokenizer.decode(t, skip_special_tokens = True, clean_up_tokenization_spaces = True) for t in y]
            
            if _%50==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            references.extend(refs)
            
    return predictions, references

In [ ]:
print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
for epoch in range(config.VAL_EPOCHS):
    predictions, references =\
        validate(epoch, t5_tokenizer, t5, device, val_loader)
    print(predictions)
    print(references)

Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe
Completed 0
["Hotels in Maharashtra will train their staff to spot signs of sex trafficking, including frequent requests for bed linen changes and 'Do not disturb' signs left on room doors for days. A mobile phone app called Rescue Me, which will allow staff to alert police of suspicious behaviour, will be developed. The initiative has been backed by the Maharashtra government.", 'The Congress party has opened a bank called \'State Bank of Tomato\' in Uttar Pradesh\'s Lucknow, in a protest against the rising prices of the vegetable. People can get interests on deposits and also get vegetables on loan in the bank. "I have deposited 0.5 Kg tomatoes, will get 1 kg after 6 months," said a man.']
['hotels in Mumbai and other Indian cities are to train their staff to spot signs of sex trafficking such as frequent requests for bed linen changes or a "Do not disturb" sign left on the door f

#### **Fine tuning, the easiest way (Trainer function)**

https://huggingface.co/transformers/training.html#trainer

##### Init

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# initialize the model architecture and weights
t5 = T5ForConditionalGeneration.from_pretrained('t5-base')

# initialize the model tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')

In [ ]:
import wandb
from os import environ

def setup_wandb(project):
    environ['WANDB_WATCH'] = 'all'
    environ['WANDB_PROJECT'] = project
    
    wandb.login()

    wandb.init(project)

    return

In [ ]:
import torch
import numpy as np

SEED = 42

TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 2

TRAIN_EPOCHS = 2
VAL_EPOCHS = 1
LEARNING_RATE = 1e-4

MAX_LEN = t5_tokenizer.model_max_length
SUMMARY_LEN = 150

# Set random seeds and deterministic pytorch for reproducibility
torch.manual_seed(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True

##### Preparing the data

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    
    def __init__(self, data, tokenizer, input_len, summ_len, eval=False):
        self.tokenizer = tokenizer
        self.data = data
        self.input_len = input_len
        self.summ_len = summ_len
        self.eval = eval
        self.article = self.data.article
        self.summary = self.data.summary
    
    def __len__(self):
        return len(self.summary)
    
    def __getitem__(self, idx):
        item = {}
        
        article = str(self.article[idx])
        article = ' '.join(article.split())

        summary = str(self.summary[idx])
        summary = ' '.join(summary.split())

        source = self.tokenizer.batch_encode_plus(
            [article],
            max_length = self.input_len,
            truncation = True,
            padding = 'max_length',
            return_tensors = 'pt')
        target = self.tokenizer.batch_encode_plus(
            [summary],
            max_length = self.summ_len,
            truncation = True,
            padding = 'max_length',
            return_tensors='pt')
        
        item['input_ids'] = source['input_ids'].squeeze()
        item['attention_mask'] = source['attention_mask'].squeeze()
        
        y = target['input_ids'].squeeze()
        if not self.eval:
            y[y == self.tokenizer.pad_token_id] = -100
        
        #item['decoder_input_ids'] = y[:-1].contiguous()
        #if not self.eval:
        #    lbl = y[1:].clone()
        #    lbl[y[1:] == self.tokenizer.pad_token_id] = -100
        #    y = lbl
        
        item['labels'] = y
        
        return item

In [ ]:
import pandas as pd

df = pd.read_csv(drive_dir.replace('\\', '')+'corpora/news_summary.csv', encoding='latin-1')
df = df[['text', 'ctext']]
df.ctext = 'summarize: ' + df.ctext
df = df.rename(columns = {'text': 'summary', 'ctext': 'article'})
#df.dropna(axis=0, inplace=True)
print(df.head())

ratio = 0.995
train_dataset = df.sample(frac = ratio, random_state = SEED)
val_dataset = df.drop(train_dataset.index).reset_index(drop = True)

train_dataset = train_dataset.sample(frac = 0.024, random_state = SEED)
train_dataset = train_dataset.reset_index(drop = True)

print("\n\nFULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(val_dataset.shape))

train_dataset = CustomDataset(
    train_dataset,
    t5_tokenizer,
    MAX_LEN,
    SUMMARY_LEN)
val_dataset = CustomDataset(
    val_dataset,
    t5_tokenizer,
    MAX_LEN,
    SUMMARY_LEN,
    eval = True)

                                             summary                                            article
0  The Administration of Union Territory Daman an...  summarize: The Daman and Diu administration on...
1  Malaika Arora slammed an Instagram user who tr...  summarize: From her special numbers to TV?appe...
2  The Indira Gandhi Institute of Medical Science...  summarize: The Indira Gandhi Institute of Medi...
3  Lashkar-e-Taiba's Kashmir commander Abu Dujana...  summarize: Lashkar-e-Taiba's Kashmir commander...
4  Hotels in Maharashtra will train their staff t...  summarize: Hotels in Mumbai and other Indian c...


FULL Dataset: (4514, 2)
TRAIN Dataset: (108, 2)
TEST Dataset: (23, 2)


##### Train

In [ ]:
from transformers import get_constant_schedule
optimizer = torch.optim.Adam(
    t5.parameters(),
    lr = LEARNING_RATE)

optimizers = (optimizer, get_constant_schedule(optimizer))

In [ ]:
wandb.login()

environ['WANDB_WATCH'] = 'all'
environ['WANDB_PROJECT'] = 'T5_fine_tuning_newssummary'

wandb: Currently logged in as: marcoabrate (use `wandb login --relogin` to force relogin)


In [ ]:
data_dir = drive_dir + 'seq2seq/xsum'
script_dir = drive_dir + 'seq2seq/finetune_trainer.py'
output_dir = '/content/drive/My\ Drive/t5_xsum_train'
log_dir = output_dir + '/logs'

!python3 $script_dir \
    --model_name_or_path t5-base \
    --tokenizer_name t5-base \
    --data_dir $data_dir \
    --task summarization \
    --max_target_length 60 \
    --val_max_target_length 60 \
    --n_train 320 \
    --n_val 80 \
    --per_device_train_batch_size $TRAIN_BATCH_SIZE \
    --per_device_eval_batch_size $VALID_BATCH_SIZE \
    --eval_beams 2 \
    --output_dir $output_dir \
    --overwrite_output_dir \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --learning_rate $LEARNING_RATE \
    --num_train_epochs 2 \
    --logging_dir $log_dir \
    --logging_steps 10 \
    --seed $SEED \
    --eval_steps 100 \
    --run_name doing_eval \
    --disable_tqdm True\
    --load_best_model_at_end \
    --metric_for_best_model rouge1 \
    --predict_with_generate

2020-10-27 09:17:33.074564: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/transformers/training_args.py:339: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,
10/27/2020 09:17:36 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/27/2020 09:17:36 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/content/drive/My Drive/t5_xsum_train', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=True, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, 

In [ ]:
from transformers import Trainer, TrainingArguments, set_seed

set_seed(SEED)

training_args = Seq2SeqTrainingArguments(
    output_dir = '/content/drive/My Drive/t5_newssummary_train',
    overwrite_output_dir = True,
    do_train = True,
    evaluation_strategy = 'steps',
    eval_steps = 10,
    #prediction_loss_only = True,
    per_device_train_batch_size = TRAIN_BATCH_SIZE,
    per_device_eval_batch_size = VALID_BATCH_SIZE,
    num_train_epochs = TRAIN_EPOCHS,
    learning_rate = LEARNING_RATE,
    logging_steps = 10,
    save_steps = 10000,
    seed = SEED,
    dataloader_num_workers = 0,
    run_name = 'doing_eval',
    logging_dir = '/content/drive/My Drive/t5_newssummary_train/logs',
    disable_tqdm = True
)

trainer = Seq2SeqTrainer(
    model = t5,
    args = training_args,
    train_dataset = train_dataset,
    #compute_metrics = my_compute_metrics,
    eval_dataset = val_dataset,
    optimizers = optimizers
)

In [ ]:
#setup_wandb('T5_fine_tuning_newssummary')
environ['WANDB_WATCH'] = 'all'
environ['WANDB_PROJECT'] = 'T5_fine_tuning_newssummary'

trainer.train()

AttributeError: ignored

In [ ]:
#trainer.save_model('/content/drive/My Drive/t5_newssummary_model')

##### Evaluate

In [ ]:
def validate(epoch, tokenizer, model, device, loader):
    
    model.eval()
    predictions = []
    references = []
    
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['labels'].to(device, dtype = torch.long)        
            input_ids = data['input_ids'].to(device, dtype = torch.long)
            attention_mask = data['attention_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = input_ids,
                attention_mask = attention_mask,
                max_length = SUMMARY_LEN,
                num_beams = 2,
                repetition_penalty = 2.5,
                length_penalty = 1.0,
                early_stopping = True)
            
            preds = [tokenizer.decode(g, skip_special_tokens = True, clean_up_tokenization_spaces = True) for g in generated_ids]
            refs = [tokenizer.decode(t, skip_special_tokens = True, clean_up_tokenization_spaces = True) for t in y]
            
            if _%50==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            references.extend(refs)
            
    return predictions, references

In [ ]:
training_args.do_train = False
training_args.do_eval = True

trainer = Trainer(
    model = t5,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    optimizers = optimizers
)

In [ ]:
from datasets import load_metric

rouge = load_metric("rouge")

In [ ]:
print('Now generating summaries on T5 for the validation dataset')
for epoch in range(VAL_EPOCHS):
    predictions, references =\
        validate(epoch, t5_tokenizer, t5, 'cuda', trainer.get_eval_dataloader())
    rouge.add_batch(predictions = predictions, references = references)

res = rouge.compute(rouge_types=['rouge1', 'rouge2', 'rougeL'], use_agregator = True)
for k in res.keys():
    print("\n", k)
    for agg in res[k]:
        print(agg)

Now generating summaries on T5 for the validation dataset
Completed 0
Completed 50
Completed 100
Completed 150
Completed 200
Completed 250
Completed 300
Completed 350
Completed 400
Completed 450

 rouge1
Score(precision=0.4864651643175072, recall=0.2772646165498224, fmeasure=0.34762771538842063)
Score(precision=0.49932275639706086, recall=0.28497676846662956, fmeasure=0.35644822151187705)
Score(precision=0.5119981095612229, recall=0.2925504943388052, fmeasure=0.3654294051443918)

 rouge2
Score(precision=0.22052402868280552, recall=0.12391624103951719, fmeasure=0.1558757283302051)
Score(precision=0.23272715383717196, recall=0.13073676452829364, fmeasure=0.1642233393742052)
Score(precision=0.24548257378871502, recall=0.13759591119440268, fmeasure=0.17276915608141485)

 rougeL
Score(precision=0.36027936082187595, recall=0.20328036961051485, fmeasure=0.2556429757587186)
Score(precision=0.3717131726256441, recall=0.20990866119417934, fmeasure=0.2636195756135158)
Score(precision=0.3833370251

In [ ]:
t5_newssummary = T5ForConditionalGeneration.from_pretrained('/content/drive/My Drive/t5_newssummary_model')

training_args.do_train = False
training_args.do_eval = True

trainer = Trainer(
    model = t5_newssummary,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    optimizers = optimizers
)

In [ ]:
print('Now generating summaries on our fine tuned model for the validation dataset')
for epoch in range(VAL_EPOCHS):
    predictions, references =\
        validate(epoch, t5_tokenizer, t5_newssummary, 'cuda', trainer.get_eval_dataloader())
    rouge.add_batch(predictions = predictions, references = references)

res = rouge.compute(rouge_types=['rouge1', 'rouge2', 'rougeL'], use_agregator = True)
for k in res.keys():
    print("\n", k)
    for agg in res[k]:
        print(agg)

Now generating summaries on our fine tuned model for the validation dataset
Completed 0
Completed 50
Completed 100
Completed 150
Completed 200
Completed 250
Completed 300
Completed 350
Completed 400
Completed 450

 rouge1
Score(precision=0.4812291218345544, recall=0.4650579755807929, fmeasure=0.46882201660160877)
Score(precision=0.49120687751148295, recall=0.4750447569299966, fmeasure=0.4783881553532816)
Score(precision=0.5014752381154781, recall=0.4858468899771374, fmeasure=0.48826799736209703)

 rouge2
Score(precision=0.2545476306853597, recall=0.24855648981001863, fmeasure=0.24919004802941577)
Score(precision=0.26633146731567536, recall=0.2599401836001135, fmeasure=0.26053623984047014)
Score(precision=0.2770684689425718, recall=0.27036750935197035, fmeasure=0.27098318129693544)

 rougeL
Score(precision=0.35621394755812186, recall=0.3448424243476525, fmeasure=0.3472603988835276)
Score(precision=0.3660402481511441, recall=0.35500057740603497, fmeasure=0.3570065267362269)
Score(precisi